
How to use Bril with real code 

Bril is very simple, very regular, ir. Bril can be extended easily.

Bril has lots of tools and examples.

lets look at a bril program.

Bril is written in JSON format.  Almost all programming languages have a way to read json.


In [76]:
import json
import subprocess
import os 

with open("tests/add.json", "r") as f:
    bril_program = json.load(f)

# print the json with nice indenting
print(json.dumps(bril_program, indent=2))


{
  "functions": [
    {
      "name": "main",
      "instrs": [
        {
          "op": "const",
          "type": "int",
          "dest": "v0",
          "value": 1
        },
        {
          "op": "const",
          "type": "int",
          "dest": "v1",
          "value": 2
        },
        {
          "op": "add",
          "type": "int",
          "dest": "v2",
          "args": [
            "v0",
            "v1"
          ]
        },
        {
          "op": "print",
          "args": [
            "v2"
          ]
        }
      ],
      "args": []
    }
  ]
}


```
{
  "functions": [
    {
      "name": "main",
      "instrs": [
        { "op": "const", "type": "int", "dest": "v0", "value": 1 },
        { "op": "const", "type": "int", "dest": "v1", "value": 2 },
        { "op": "add", "type": "int", "dest": "v2",
          "args": ["v0", "v1"] },
        { "op": "print", "args": ["v2"] }
      ],
      "args": []
    }
  ]
}
```



There is also a text form  which is more readable

we have program ***bril2txt*** and ***bril2json*** that make it easy to convert. Keep in mind that the json format is Bril and thats where you will do all the work. 


In [77]:
import os 
os.system("bril2txt < tests/add.json")

@main {
  v0: int = const 1;
  v1: int = const 2;
  v2: int = add v0 v1;
  print v2;
}


0

In [84]:
!bril2txt < tests/add.json

@main {
  v0: int = const 1;
  v1: int = const 2;
  v2: int = add v0 v1;
  print v2;
}


In [78]:
# you can connect tools via pipes 
os.system("bril2txt < tests/add.json | bril2json")

{
  "functions": [
    {
      "instrs": [
        {
          "dest": "v0",
          "op": "const",
          "type": "int",
          "value": 1
        },
        {
          "dest": "v1",
          "op": "const",
          "type": "int",
          "value": 2
        },
        {
          "args": [
            "v0",
            "v1"
          ],
          "dest": "v2",
          "op": "add",
          "type": "int"
        },
        {
          "args": [
            "v2"
          ],
          "op": "print"
        }
      ],
      "name": "main"
    }
  ]
}


0

Bril tools were mostly student projects,  as you think about your projects, you might consider adding a new tool.
you can setup Bril on your local linux (wsl) machine by cloning the bril github and installing all the tools




In [79]:
# Bril has an interpreter which reads in json and output the result

os.system(f'brili <tests/add.json')


3


0

Lets write a sample program - generating the cfg 

I'll do this in two steps
1) find all the basic blocks
2) add all the cfg edges 

finding the basic blocks from a list of instructions-  
keep adding instructions till we get to a terminator or a label
(do we add labels?)
```
in: list of instrs 
out: list of lists of instrs 

blocks = []
curr_block = []
for each instr in list 
   if the instruction is not a label put it on curr_block
   if instr is a label or terminator 
      put curr_block on blocks
      curr_block = []

if curr_block is not empty add it to blocks
return blocks 
```
```
find cfg: in is bril progmam in json 
for each function find the list of instructions 
    get last_instr 
    if it is a terminator  br/jmp/ret  
    --- what do we want to do with call? 
    else it is a fall through
```
we need a map (block_map) label->block so we can add edges for blocks that end in br/jmp - can build this while getting the blocks or we can put the label as 
the first instruction

how do we get fall through? 

what about a return

if every block ends with a terminator, and every block has a label, then no fall through case 
what happens if try to delete the terminator (because the block never executes)

I'll use a python data structure called OrderedDict, when you iterate over the items in a ordered dict,  they come back in the order that they were installed.



In [80]:
# playing around with hacking- I'll use a generator 


#Instructions that terminate a basic block.
TERMINATORS = 'br', 'jmp', 'ret'


def form_blocks(instrs):
    """Given a list of Bril instructions, generate a sequence of
    instruction lists representing the basic blocks in the program.

    Every instruction in `instr` will show up in exactly one block. Jump
    and branch instructions may only appear at the end of a block, and
    control can transfer only to the top of a basic block---so labels
    can only appear at the *start* of a basic block. Basic blocks may
    not be empty.
    """

    # Start with an empty block.
    cur_block = []

    for instr in instrs:
        if 'op' in instr:  # It's an instruction.
            # Add the instruction to the currently-being-formed block.
            cur_block.append(instr)

            # If this is a terminator (branching instruction), it's the
            # last instruction in the block. Finish this block and
            # start a new one.
            if instr['op'] in TERMINATORS:
                yield cur_block
                cur_block = []

        else:  # It's a label.
            # End the block here (if it contains anything).
            if cur_block:
                yield cur_block

            # Start a new block with the label.
            cur_block = [instr]

    # Produce the final block, if any.
    if cur_block:
        yield cur_block


In [81]:
def print_blocks(bril):
    """Given a Bril program, print out its basic blocks.
    """


    func = bril['functions'][0]  # We only process one function.
    for block in form_blocks(func['instrs']):
        # Mark the block.
        leader = block[0]
        if 'label' in leader:
            print( f"block {leader['label']}")
            block = block[1:]  # Hide the label, for concision.
        else:
            print('anonymous block:')

        # Print the instructions.
        for instr in block:
            print(instr)


print_blocks(bril_program)

anonymous block:
{'op': 'const', 'type': 'int', 'dest': 'v0', 'value': 1}
{'op': 'const', 'type': 'int', 'dest': 'v1', 'value': 2}
{'op': 'add', 'type': 'int', 'dest': 'v2', 'args': ['v0', 'v1']}
{'op': 'print', 'args': ['v2']}


In [82]:
with open("tests/jmp.bril", 'r') as f:
          test2 = f.read()

print(test2)


result = subprocess.check_output('bril2json <tests/jmp.bril', shell=True)

test2json = json.loads(result)
print(test2json)

@main {
  v: int = const 4;
  jmp .somewhere;
  v: int = const 2;
.somewhere:
  print v;
}

{'functions': [{'instrs': [{'dest': 'v', 'op': 'const', 'type': 'int', 'value': 4}, {'labels': ['somewhere'], 'op': 'jmp'}, {'dest': 'v', 'op': 'const', 'type': 'int', 'value': 2}, {'label': 'somewhere'}, {'args': ['v'], 'op': 'print'}], 'name': 'main'}]}


In [83]:
print_blocks(test2json['functions'][0])

KeyError: 'functions'

In [ ]:
# now for the map 
from collections import OrderedDict


def block_map(blocks):
    """Given a sequence of basic blocks, which are lists of instructions,
    produce a `OrderedDict` mapping names to blocks.

    The name of the block comes from the label it starts with, if any.
    Anonymous blocks, which don't start with a label, get an
    automatically generated name. Blocks in the mapping have their
    labels removed.
    """
    by_name = OrderedDict()

    for block in blocks:
        # Generate a name for the block.
        if 'label' in block[0]:
            # The block has a label. Remove the label but use it for the
            # block's name.
            name = block[0]['label']
            block = block[1:]
        else:
            # Make up a new name for this anonymous block.
            name = f'gen_bk_{len(by_name)}'

        # Add the block to the mapping.
        by_name[name] = block

    return by_name


blks = form_blocks(test2json['functions'][0]['instrs'])
od = block_map(blks)
for (name, instrs) in od.items():
    print (name, instrs)

gen_bk_0 [{'dest': 'v', 'op': 'const', 'type': 'int', 'value': 4}, {'labels': ['somewhere'], 'op': 'jmp'}]
gen_bk_1 [{'dest': 'v', 'op': 'const', 'type': 'int', 'value': 2}]
somewhere [{'args': ['v'], 'op': 'print'}]


# finally the cfg

~~~
out cfg = {} map label -> list of labels the successors of the block

for i , block in enumerate(blocks)  # blocks is a ordereddict 
  last = block[i]  # last instruction
  if last is jmp:
     cfg[block_name] = jmp.dest
  elif last is br:
    cfg[block.name] = [ last.if_label, last.else_label]
  else
     # fall through
    cfg[block_name = blocks[i+1].name  ## special case for last block
~~~

In [ ]:
def get_cfg(ordered_blocks):
    cfg = {}

    labels = list(ordered_blocks.keys())

    for i, (block_name, block) in enumerate(ordered_blocks.items()):
        last = block[-1]
        op = last['op']

        if op == 'jmp':
            cfg[block_name] = last['labels']
        elif op == 'br':
            cfg[block_name] = last['labels']
        else:
            if i+1 < len(labels):  # last block does not fall through
                cfg[block_name] = [labels[i+1]]
    return cfg


blks = form_blocks(test2json['functions'][0]['instrs'])
od = block_map(blks)
cfg = get_cfg(od)

print(cfg)

{'gen_bk_0': ['somewhere'], 'gen_bk_1': ['somewhere']}
